In [1]:
# Import our dependencies
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs('checkpoints/', exist_ok = True)
checkpoint_path = 'checkpoints/weights.{epoch:02d}.hdf5'
data = pd.read_csv('coin_Bitcoin.csv')

In [22]:
def run_lstm_model (data, target, drop = [], thresholds = [], window = 7,
        input_node = { 'units': 80, 'activation': 'relu' },
        hidden_nodes = [
            { 'units': 50, 'activation': 'relu' }
        ],
        output_node = { 'units': 1, 'activation': 'relu' },
        optimizer = 'adam',
        epochs = 100, save_path = '', seed = 0, verbose = 0):
    """
    Processes data, creates a neuro network, trains, and tests it.  Returns the trained model.
    data: target and features values in a data frame
    target: the name of the y col in data
    drop: cols to drop from the data
    thresholds: an array of obects with properties 'col' and 'threshold'
        col is the column to bin
        threshold is teh value below which all vals should be binned into other
    target: the name of the target col
    input_node: an object with properties 'units' and 'activation'
    hidden_nodes: an array of objects with properties 'units' and 'activation'
    output_node: an object with properties 'units' and 'activation'
        units is an int
        activation is a string in: relu, leaky_relu, tanh, sigmoid
    optimizer: a string in: adadelta, adagrad, adam, adamax, ftrl, optimiser, rmsprop, sgd
    epochs: how long to train the model
    save_path: where to save epoch weights; an empty string tells the code to not save epochs 
    seed: an int to seed the train/test split on
    verbose: an int to set how much data the model outputs while running
    """
    
    # clone data
    data_clone = data.copy()
    
    # clean data
    data_clone = clean_data(data_clone, drop, thresholds)
    
    #get train/test data and make model
    X_train, X_test, y_train, y_test = get_target_and_features(data_clone, target, seed, window)
    
    model = create_model(X_train, input_node, hidden_nodes, output_node, optimizer)
    
    #run and evaluate model; return model
    return evaluate(model, X_train, X_test, y_train, y_test, epochs, save_path, verbose)
    


In [23]:
def clean_data(data, drop, thresholds):
    # drop columns
    data = data.drop(drop, 1)
    
    # compress large sets of data
    for threshold in thresholds:
        data = compress(data, threshold['col'], threshold['threshold'])
        
    ####TODO
    # We need code to handle the date data appropriately
    
    #replace cats as dummys
    return encode(data)

In [24]:
def compress (data, col, threshold):
    
    # get vals to replace
    counts = data[col].value_counts()
    replace = list(counts[counts < threshold].index)
    
    #replace the values and return
    for rep in replace:
        data[col] = data[col].replace(rep, 'Other')
    return data
    

In [25]:
def encode(data):    
    # get cats     
    cat = data.dtypes[data.dtypes == 'object'].index.tolist()
    
    # make encoder and encode
    enc = OneHotEncoder(sparse = False)
    encode = pd.DataFrame(enc.fit_transform(data[cat]))
    encode.columns = enc.get_feature_names(cat)    
    
    # return new df
    return data.merge(encode, left_index = True, right_index = True).drop(cat, 1)


In [26]:
def get_target_and_features(data, target, seed, window):
    # Split our preprocessed data into our features and target arrays
    y = data[target].values
    X = data.drop([target], 1).values

    # Split the preprocessed data into a training and testing dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = seed)
    
    # Create a StandardScaler instances
    scaler = StandardScaler()

    # Fit the StandardScaler
    X_scaler = scaler.fit(X_train)

    # Scale the data
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)
    
    # create window increments     
    X_train, y_train = set_window(X_train, y_train, window)
    X_test, y_test = set_window(X_test, y_test, window)
   
    
    return X_train, X_test, y_train, y_test

In [27]:
def set_window(X, y, window):
    #Add window
    Xin = []
    next_y = []
    for i in range(window, len(X)):
        try:
            next_y.append(y[i])
            Xin.append(X[i-window:i])
        # for the cases where there is no future y, just drop the data.         
        except:
            pass
        
    # Reshape data to format for LSTM
    X, y = np.array(Xin), np.array(next_y)
    X = X.reshape(X.shape[0], X.shape[1], X.shape[2])
    
    return X, y
    

In [28]:
def create_model(X_train, input_node, hidden_nodes, output_node, optimizer):

    # get the input node number
    input_shape = (X_train.shape[1], X_train.shape[2])
    
    # create model
    nn = tf.keras.models.Sequential()
    
    #add layers
    #input
    nn.add(tf.keras.layers.LSTM(units = input_node['units'], input_shape = input_shape,
                                 activation = input_node['activation'], return_sequences = True))
    nn.add(tf.keras.layers.Dropout(.2))
        
    #hidden
    for ind, node in enumerate(hidden_nodes):
        if ind == len(hidden_nodes) - 1:
            nn.add(tf.keras.layers.LSTM(units = node['units'], activation = node['activation']))
        else:
            nn.add(tf.keras.layers.LSTM(units = node['units'], activation = node['activation'],
                                        return_sequences = True))
        nn.add(tf.keras.layers.Dropout(.2))
    
    #output
    nn.add(tf.keras.layers.Dense(units = output_node['units'], activation = output_node['activation']))

    #compile and return
    nn.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    return nn


In [29]:
def create_callback (path, verbose):
    return ModelCheckpoint(
        filepath = path,
        verbose = verbose,
        save_weights_only = True,
        save_freq = 'epoch')

In [30]:
def evaluate(model, X_train, X_test, y_train, y_test, epochs, save_path, verbose):
    # it either wiith callpaths to save epochs or without
    if save_path != '':
        fit_model = model.fit(X_train, y_train, epochs = epochs, verbose = verbose,
                              callbacks = [create_callback(save_path, verbose)])
    else:
        fit_model = model.fit(X_train, y_train, epochs = epochs, verbose = verbose)
    
    #give feedback on model performance
    model_loss, model_accuracy = model.evaluate(X_test, y_test, verbose = verbose)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
    
    #return the model
    return model

In [31]:
model = run_lstm_model(data, 'Close', drop = ['Name', 'Symbol', 'SNo', 'Date'])

Loss: -60962.484375, Accuracy: 0.0
